# Исследовательский анализ данных о качестве вина

В этом ноутбуке мы проведем анализ набора данных о качестве красного вина и исследуем взаимосвязи между химическими свойствами вина и его качеством.

In [ ]:
# Импорт необходимых библиотек
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Настройка визуализации
plt.style.use('seaborn-whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
sns.set_palette('viridis')

# Отключение предупреждений
import warnings
warnings.filterwarnings('ignore')

## 1. Загрузка данных

In [ ]:
# Загрузка данных
wine_data = pd.read_csv('../data/winequality-red.csv', sep=';')

# Просмотр первых строк данных
wine_data.head()

In [ ]:
# Проверка размера набора данных
print(f"Размер набора данных: {wine_data.shape[0]} строк и {wine_data.shape[1]} столбцов")

In [ ]:
# Проверка типов данных и пропущенных значений
wine_data.info()

In [ ]:
# Статистическое описание данных
wine_data.describe()

## 2. Анализ распределения качества вина

In [ ]:
# Распределение оценок качества вина
plt.figure(figsize=(10, 6))
ax = sns.countplot(x='quality', data=wine_data, palette='viridis')
plt.title('Распределение оценок качества вина', fontsize=16)
plt.xlabel('Оценка качества', fontsize=14)
plt.ylabel('Количество образцов', fontsize=14)

# Добавление подписей с количеством над столбцами
for p in ax.patches:
    ax.annotate(f'{p.get_height()}', 
                (p.get_x() + p.get_width() / 2., p.get_height()), 
                ha = 'center', va = 'bottom', fontsize=12)

plt.show()

In [ ]:
# Процентное соотношение оценок качества
quality_counts = wine_data['quality'].value_counts(normalize=True) * 100
print("Процентное соотношение оценок качества:")
for quality, percentage in quality_counts.sort_index().items():
    print(f"Оценка {quality}: {percentage:.2f}%")

## 3. Анализ корреляций между характеристиками вина

In [ ]:
# Расчет матрицы корреляций
corr_matrix = wine_data.corr()

# Визуализация матрицы корреляций
plt.figure(figsize=(14, 10))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title('Матрица корреляций между характеристиками вина', fontsize=16)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# Выделение наиболее коррелированных с качеством характеристик
quality_correlations = corr_matrix['quality'].sort_values(ascending=False)
print("Корреляция характеристик с качеством вина:")
print(quality_correlations)

## 4. Визуализация взаимосвязей между характеристиками и качеством

In [ ]:
# Функция для создания графика взаимосвязи между характеристикой и качеством
def plot_feature_vs_quality(feature):
    plt.figure(figsize=(12, 6))
    
    # Boxplot
    plt.subplot(1, 2, 1)
    sns.boxplot(x='quality', y=feature, data=wine_data)
    plt.title(f'Boxplot: {feature} vs Quality', fontsize=14)
    plt.xlabel('Quality', fontsize=12)
    plt.ylabel(feature, fontsize=12)
    
    # Violin plot
    plt.subplot(1, 2, 2)
    sns.violinplot(x='quality', y=feature, data=wine_data, inner='quartile')
    plt.title(f'Violin Plot: {feature} vs Quality', fontsize=14)
    plt.xlabel('Quality', fontsize=12)
    plt.ylabel(feature, fontsize=12)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Визуализация взаимосвязи между содержанием алкоголя и качеством
plot_feature_vs_quality('alcohol')

In [ ]:
# Визуализация взаимосвязи между летучей кислотностью и качеством
plot_feature_vs_quality('volatile acidity')

In [ ]:
# Визуализация взаимосвязи между сульфатами и качеством
plot_feature_vs_quality('sulphates')

## 5. Распределения характеристик вина

In [ ]:
# Визуализация распределений всех характеристик
features = wine_data.columns.drop('quality')
n_features = len(features)
n_rows = (n_features + 1) // 2

fig, axes = plt.subplots(n_rows, 2, figsize=(16, 4 * n_rows))
axes = axes.flatten()

for i, feature in enumerate(features):
    sns.histplot(wine_data[feature], kde=True, ax=axes[i])
    axes[i].set_title(f'Распределение {feature}', fontsize=14)
    axes[i].set_xlabel(feature, fontsize=12)
    axes[i].set_ylabel('Частота', fontsize=12)

# Скрыть пустые подграфики, если их количество нечетное
if n_features % 2 != 0:
    axes[-1].set_visible(False)

plt.tight_layout()
plt.show()

## 6. Парные графики для наиболее важных характеристик

In [ ]:
# Выбор наиболее важных характеристик на основе корреляции с качеством
top_features = quality_correlations.index[:5]  # Топ-5 характеристик
top_features = top_features.tolist()
if 'quality' in top_features:
    top_features.remove('quality')
top_features.append('quality')

# Создание парных графиков
plt.figure(figsize=(15, 10))
sns.pairplot(wine_data[top_features], hue='quality', palette='viridis', diag_kind='kde', plot_kws={'alpha': 0.6})
plt.suptitle('Парные графики для наиболее важных характеристик', y=1.02, fontsize=16)
plt.tight_layout()
plt.show()

## 7. Анализ выбросов

In [ ]:
# Функция для обнаружения выбросов с использованием метода IQR
def detect_outliers(df, feature):
    Q1 = df[feature].quantile(0.25)
    Q3 = df[feature].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[feature] < lower_bound) | (df[feature] > upper_bound)]
    return outliers, lower_bound, upper_bound

In [ ]:
# Анализ выбросов для каждой характеристики
for feature in features:
    outliers, lower_bound, upper_bound = detect_outliers(wine_data, feature)
    outlier_percentage = len(outliers) / len(wine_data) * 100
    
    print(f"Характеристика: {feature}")
    print(f"Нижняя граница: {lower_bound:.2f}, Верхняя граница: {upper_bound:.2f}")
    print(f"Количество выбросов: {len(outliers)} ({outlier_percentage:.2f}%)")
    print("-" * 50)

## 8. Подготовка данных для моделирования

In [ ]:
# Разделение данных на признаки и целевую переменную
X = wine_data.drop('quality', axis=1)
y = wine_data['quality']

# Проверка баланса классов
plt.figure(figsize=(10, 6))
sns.countplot(x=y, palette='viridis')
plt.title('Распределение целевой переменной (качество вина)', fontsize=16)
plt.xlabel('Качество', fontsize=14)
plt.ylabel('Количество образцов', fontsize=14)
plt.show()

## 9. Выводы

На основе проведенного исследовательского анализа данных можно сделать следующие выводы:

1. Набор данных содержит информацию о химических свойствах красного вина и его качестве.
2. Качество вина оценивается по шкале от 3 до 8, с преобладанием средних оценок (5 и 6).
3. Наиболее сильные корреляции с качеством вина имеют следующие характеристики:
   - Содержание алкоголя (положительная корреляция)
   - Летучая кислотность (отрицательная корреляция)
   - Содержание сульфатов (положительная корреляция)
4. В данных присутствуют выбросы, которые могут повлиять на качество модели.
5. Распределение целевой переменной несбалансировано, что необходимо учесть при моделировании.

Эти выводы будут использованы при построении модели для предсказания качества вина.